#### Turn a dialogue into State->action->state->action ....-> terminal state, reward

In [242]:
import glob
import pandas as pd
import numpy as np
from textblob import TextBlob
import pickle

In [243]:
def get_state(utter):
    """ Question, Sentiment, Subjectivity, Length"""
    utter = utter.split(":")[-1].strip("\n")
    state = []
    state.append(str(int(is_question(utter))))
    state.append(str(int(is_positive(utter))))
    state.append(str(int(is_subjective(utter))))
    state.append(str(int(is_long(utter))))
    return state
    
def is_question(utter):
    return utter[-1] == "?"

def is_positive(utter):
    senti = TextBlob(utter).sentiment.polarity
    return senti > 0
    
def is_subjective(utter):
    sub = TextBlob(utter).sentiment.subjectivity
    return sub > 0
    
def is_long(utter):
    return len(utter) > 20

def set_reward(df, lastline):
    ratings = lastline.strip("\n").strip(",").split(",")
    for rating in ratings:
        category, score = rating.split("=")
        df.set_value(df.index[-1], category, int(score))
    return df

In [244]:
filenames = glob.glob("../data/300_convo/*")
parsed_dict = {}
for filename in filenames:
    lines = open(filename, "r", errors="replace").readlines()
    df_sa = pd.DataFrame(columns=["Start", "Question", "Sentiment", "Subjectivity", "Length", 
                                  "Action", "overall", "start", "interupt", "engaing", "return"])
    bot_lines = [i for i in lines[1:] if "Bot_" in i]
    user_lines = [i for i in lines if "_None_" in i]

    # initial state/action/reward
    df_sa.loc[0] = ["1","0","0","0","0",lines[0].split("_")[1],0,0,0,0,0] 

    # intermediate state/action/reward
    for i, (user, bot) in enumerate(zip(user_lines[:-1], bot_lines)):
        df_sa.loc[i+1] = ["0"] + get_state(user) + [bot.split("_")[1]] + [0]*5
        
    # terminal state/reward
    df_sa.loc[i+2] = ["T"]*5 + ["None"] + [0]*5
    df_sa = set_reward(df_sa.copy(), lines[-1])
    
    parsed_dict[filename] = df_sa

In [245]:
pickle.dump(parsed_dict, open("../data/parsed_data.pkl", "wb"))

In [246]:
print(open(filename, "r").read())

Bot_elaborate_20:56:22:You gotta tell me more.
Yue Cao_None_20:56:31:blablablablablablabla
Bot_elaborate_20:56:31:Can you elaborate on that?
Yue Cao_None_20:56:37:Yes
Bot_neutral_20:56:37:haha
Yue Cao_None_20:56:40:haha
Bot_elaborate_20:56:40:Huh?
Yue Cao_None_20:56:44:Huh?
Bot_neutral_20:56:44:Well okay.
Yue Cao_None_20:56:47:Good
Bot_neutral_20:56:47:LOL
Yue Cao_None_20:56:50:LOL
Bot_affirmative_20:56:50:That's hilarious!
Yue Cao_None_20:56:58:That's hilarious
Bot_opinion_20:56:58:Nothing isn't better or worse than anything. Nothing is just nothing.
Yue Cao_None_20:57:14:You a right
Bot_neutral_20:57:14:Really?
Yue Cao_None_20:57:20:right
overall=2,start=2,interupt=2,engaing=4,return=3,


In [247]:
df_sa

,Start,Question,Sentiment,Subjectivity,Length,Action,overall,start,interupt,engaing,return
0,1,0,0,0,0,elaborate,0,0,0,0,0
1,0,0,0,0,1,elaborate,0,0,0,0,0
2,0,0,0,0,0,neutral,0,0,0,0,0
3,0,0,1,1,0,elaborate,0,0,0,0,0
4,0,1,0,0,0,neutral,0,0,0,0,0
5,0,0,1,1,0,neutral,0,0,0,0,0
6,0,0,1,1,0,affirmative,0,0,0,0,0
7,0,0,1,1,0,opinion,0,0,0,0,0
8,0,0,1,1,0,neutral,0,0,0,0,0
9,T,T,T,T,T,None,2,2,2,4,3


In [265]:
print(open("../data/300_convo/wei_2017-05-05_10.txt", "r", errors="ignore").read())

Bot_question_21:26:13:What plot would you want to happen differently?
wei_None_21:27:39:I don't know too much about the TV series plots.
Bot_elaborate_21:27:39:You gotta tell me more.
wei_None_21:27:50:I just read the books, and I love it.
Bot_affirmative_21:27:50:I know what you mean
wei_None_21:28:22:how long does it take you to finish all the books?
Bot_opinion_21:28:22:Dracarys
wei_None_21:30:07:that's so cool to train the dragons.
Bot_opinion_21:30:07:Life is not a song, sweetling. Someday you may learn that, to your sorrow.
wei_None_21:31:51:that's what about real life, it sucks, but you gonna love it.
Bot_affirmative_21:31:51:You have a great piont point.
wei_None_21:31:54:thatnks
overall=5,start=5,interupt=5,engaing=4,return=5,


In [263]:
def find_examples(dialogs):
    for filename, df in dialogs.items():
        if len(df.index) < 8 and df.iloc[-1]["overall"] > 3 and "si chang" not in filename:
            return filename, df

In [264]:
print(find_examples(parsed_dict))

('../data/300_convo/wei_2017-05-05_10.txt',   Start Question Sentiment Subjectivity Length       Action overall start  \
0     1        0         0            0      0     question       0     0   
1     0        0         1            1      1    elaborate       0     0   
2     0        0         1            1      1  affirmative       0     0   
3     0        1         0            1      1      opinion       0     0   
4     0        0         1            1      1      opinion       0     0   
5     0        0         1            1      1  affirmative       0     0   
6     T        T         T            T      T         None       5     5   

  interupt engaing return  
0        0       0      0  
1        0       0      0  
2        0       0      0  
3        0       0      0  
4        0       0      0  
5        0       0      0  
6        5       4      5  )


In [266]:
parsed_dict["../data/300_convo/wei_2017-05-05_10.txt"]

,Start,Question,Sentiment,Subjectivity,Length,Action,overall,start,interupt,engaing,return
0,1,0,0,0,0,question,0,0,0,0,0
1,0,0,1,1,1,elaborate,0,0,0,0,0
2,0,0,1,1,1,affirmative,0,0,0,0,0
3,0,1,0,1,1,opinion,0,0,0,0,0
4,0,0,1,1,1,opinion,0,0,0,0,0
5,0,0,1,1,1,affirmative,0,0,0,0,0
6,T,T,T,T,T,None,5,5,5,4,5
